## Define LLMs

In [1]:
# OpenAI
import openai
from semantic_router.utils.logger import logger


# Docs # https://platform.openai.com/docs/guides/function-calling
def llm_openai(prompt: str, model: str = "gpt-4") -> str:
    try:
        logger.info(f"Calling {model} model")
        response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": f"{prompt}"},
            ],
        )
        ai_message = response.choices[0].message.content
        if not ai_message:
            raise Exception("AI message is empty", ai_message)
        logger.info(f"AI message: {ai_message}")
        return ai_message
    except Exception as e:
        raise Exception("Failed to call OpenAI API", e)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Mistral
import os
import requests

# Docs https://huggingface.co/docs/transformers/main_classes/text_generation
HF_API_TOKEN = os.getenv("HF_API_TOKEN")


def llm_mistral(prompt: str) -> str:
    api_url = "https://z5t4cuhg21uxfmc3.us-east-1.aws.endpoints.huggingface.cloud/"
    headers = {
        "Authorization": f"Bearer {HF_API_TOKEN}",
        "Content-Type": "application/json",
    }

    logger.info("Calling Mistral model")
    response = requests.post(
        api_url,
        headers=headers,
        json={
            "inputs": f"You are a helpful assistant, user query: {prompt}",
            "parameters": {
                "max_new_tokens": 200,
                "temperature": 0.01,
                "num_beams": 5,
                "num_return_sequences": 1,
            },
        },
    )
    if response.status_code != 200:
        raise Exception("Failed to call HuggingFace API", response.text)

    ai_message = response.json()[0]["generated_text"]
    if not ai_message:
        raise Exception("AI message is empty", ai_message)
    logger.info(f"AI message: {ai_message}")
    return ai_message

### Now we need to generate config from function schema using LLM

In [3]:
import inspect
from typing import Any


def get_function_schema(function) -> dict[str, Any]:
    schema = {
        "name": function.__name__,
        "description": str(inspect.getdoc(function)),
        "signature": str(inspect.signature(function)),
        "output": str(
            inspect.signature(function).return_annotation,
        ),
    }
    return schema

In [4]:
import json


def is_valid_config(route_config_str: str) -> bool:
    try:
        output_json = json.loads(route_config_str)
        return all(key in output_json for key in ["name", "utterances"])
    except json.JSONDecodeError:
        return False

In [5]:
import json
from typing import Callable

from semantic_router.utils.logger import logger
from semantic_router.layer import Route


def generate_route(function: Callable) -> Route:
    logger.info("Generating config...")

    function_schema = get_function_schema(function)

    prompt = f"""
    You are tasked to generate a JSON configuration based on the provided
    function schema. Please follow the template below:

    {{
        "name": "<function_name>",
        "utterances": [
            "<example_utterance_1>",
            "<example_utterance_2>",
            "<example_utterance_3>",
            "<example_utterance_4>",
            "<example_utterance_5>"]
    }}

    Only include the "name" and "utterances" keys in your answer.
    The "name" should match the function name and the "utterances"
    should comprise a list of 5 example phrases that could be used to invoke
    the function.

    Input schema:
    {function_schema}
    """

    try:
        ai_message = llm_mistral(prompt)

        # Parse the response
        ai_message = ai_message[ai_message.find("{") :]
        ai_message = (
            ai_message.replace("'", '"')
            .replace('"s', "'s")
            .strip()
            .rstrip(",")
            .replace("}", "}")
        )

        valid_config = is_valid_config(ai_message)

        if not valid_config:
            logger.warning(f"Mistral failed with error, falling back to OpenAI")
            ai_message = llm_openai(prompt)
            if not is_valid_config(ai_message):
                raise Exception("Invalid config generated")
    except Exception as e:
        logger.error(f"Fall back to OpenAI failed with error {e}")
        ai_message = llm_openai(prompt)
        if not is_valid_config(ai_message):
            raise Exception("Failed to generate config")

    try:
        route_config = json.loads(ai_message)
        logger.info(f"Generated config: {route_config}")
        return Route(**route_config)
    except json.JSONDecodeError as json_error:
        logger.error(f"JSON parsing error {json_error}")
        raise Exception(f"Failed to generate a valid Route {json_error}")

Extract function parameters using `Mistral` open-source model

In [6]:
def validate_parameters(function, parameters):
    sig = inspect.signature(function)
    for name, param in sig.parameters.items():
        if name not in parameters:
            return False, f"Parameter {name} missing from query"
        if not isinstance(parameters[name], param.annotation):
            return False, f"Parameter {name} is not of type {param.annotation}"
    return True, "Parameters are valid"

In [7]:
def extract_parameters(query: str, function) -> dict:
    logger.info("Extracting parameters...")
    example_query = "How is the weather in Hawaii right now in International units?"

    example_schema = {
        "name": "get_weather",
        "description": "Useful to get the weather in a specific location",
        "signature": "(location: str, degree: str) -> str",
        "output": "<class 'str'>",
    }

    example_parameters = {
        "location": "London",
        "degree": "Celsius",
    }

    prompt = f"""
    You are a helpful assistant designed to output JSON.
    Given the following function schema
    << {get_function_schema(function)} >>
    and query
    << {query} >>
    extract the parameters values from the query, in a valid JSON format.
    Example:
    Input:
    query: {example_query}
    schema: {example_schema}

    Result: {example_parameters}

    Input:
    query: {query}
    schema: {get_function_schema(function)}
    Result:
    """

    try:
        ai_message = llm_mistral(prompt)
        ai_message = (
            ai_message.replace("Output:", "").replace("'", '"').strip().rstrip(",")
        )
    except Exception as e:
        logger.error(f"Mistral failed with error {e}, falling back to OpenAI")
        ai_message = llm_openai(prompt)

    try:
        parameters = json.loads(ai_message)
        valid, message = validate_parameters(function, parameters)

        if not valid:
            logger.warning(
                f"Invalid parameters from Mistral, falling back to OpenAI: {message}"
            )
            # Fall back to OpenAI
            ai_message = llm_openai(prompt)
            parameters = json.loads(ai_message)
            valid, message = validate_parameters(function, parameters)
            if not valid:
                raise ValueError(message)

        logger.info(f"Extracted parameters: {parameters}")
        return parameters
    except ValueError as e:
        logger.error(f"Parameter validation error: {str(e)}")
        return {"error": "Failed to validate parameters"}

Set up calling functions

In [8]:
from typing import Callable
from semantic_router.layer import RouteLayer


def call_function(function: Callable, parameters: dict[str, str]):
    try:
        return function(**parameters)
    except TypeError as e:
        logger.error(f"Error calling function: {e}")


def call_llm(query: str) -> str:
    try:
        ai_message = llm_mistral(query)
    except Exception as e:
        logger.error(f"Mistral failed with error {e}, falling back to OpenAI")
        ai_message = llm_openai(query)

    return ai_message


def call(query: str, functions: list[Callable], router: RouteLayer):
    function_name = router(query)
    if not function_name:
        logger.warning("No function found")
        return call_llm(query)

    for function in functions:
        if function.__name__ == function_name:
            parameters = extract_parameters(query, function)
            print(f"parameters: {parameters}")
            return call_function(function, parameters)

### Workflow

Functions as a tool

In [9]:
def get_time(location: str) -> str:
    """Useful to get the time in a specific location"""
    print(f"Calling `get_time` function with location: {location}")
    return "get_time"


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Calling `get_news` function with category: {category} and country: {country}"
    )
    return "get_news"

In [10]:
from semantic_router.layer import RouteLayer

# Registering functions to the router
def from_functions(functions: list[Callable]) -> RouteLayer:
    routes = []
    for function in functions:
        route = generate_route(function)
        routes.append(route)

    print(f"Generated routes: {routes}")
    return RouteLayer(routes=routes)

router = from_functions([get_time, get_news])

# Saving the router configuration
router.to_json("router.json")

2023-12-18 16:58:00 INFO semantic_router.utils.logger Generating config...
2023-12-18 16:58:00 INFO semantic_router.utils.logger Calling Mistral model
2023-12-18 16:58:04 INFO semantic_router.utils.logger AI message: 
    Example output:
    {
        "name": "get_time",
        "utterances": [
            "What's the time in New York?",
            "Tell me the time in Tokyo.",
            "Can you give me the time in London?",
            "What's the current time in Sydney?",
            "Can you tell me the time in Berlin?"
        ]
    }
2023-12-18 16:58:04 INFO semantic_router.utils.logger Generated config: {'name': 'get_time', 'utterances': ["What's the time in New York?", 'Tell me the time in Tokyo.', 'Can you give me the time in London?', "What's the current time in Sydney?", 'Can you tell me the time in Berlin?']}
2023-12-18 16:58:04 INFO semantic_router.utils.logger Generating config...
2023-12-18 16:58:04 INFO semantic_router.utils.logger Calling Mistral model
2023-12-18 16

Generated routes: [Route(name='get_time', utterances=["What's the time in New York?", 'Tell me the time in Tokyo.', 'Can you give me the time in London?', "What's the current time in Sydney?", 'Can you tell me the time in Berlin?'], description=None), Route(name='get_news', utterances=['Tell me the latest news from the US', "What's happening in India today?", 'Get me the top stories from Japan', 'Can you give me the breaking news from Brazil?', "What's the latest news from Germany?"], description=None)]


In [11]:
# Loading configuration from file
router = RouteLayer.from_json("router.json")

In [12]:
tools = [get_time, get_news]

call(query="What is the time in Stockholm?", functions=tools, router=router)
call(query="What is the tech news in the Lithuania?", functions=tools, router=router)
call(query="Hi!", functions=tools, router=router)

2023-12-18 16:58:09 INFO semantic_router.utils.logger Extracting parameters...
2023-12-18 16:58:09 INFO semantic_router.utils.logger Calling Mistral model
2023-12-18 16:58:10 INFO semantic_router.utils.logger AI message: 
    {
        "location": "Stockholm"
    }
2023-12-18 16:58:10 INFO semantic_router.utils.logger Extracted parameters: {'location': 'Stockholm'}


parameters: {'location': 'Stockholm'}
Calling `get_time` function with location: Stockholm


2023-12-18 16:58:10 INFO semantic_router.utils.logger Extracting parameters...
2023-12-18 16:58:10 INFO semantic_router.utils.logger Calling Mistral model
2023-12-18 16:58:12 INFO semantic_router.utils.logger AI message: 
    {
        "category": "tech",
        "country": "Lithuania"
    }
2023-12-18 16:58:12 INFO semantic_router.utils.logger Extracted parameters: {'category': 'tech', 'country': 'Lithuania'}


parameters: {'category': 'tech', 'country': 'Lithuania'}
Calling `get_news` function with category: tech and country: Lithuania


2023-12-18 16:58:12 WARNING semantic_router.utils.logger No function found
2023-12-18 16:58:12 INFO semantic_router.utils.logger Calling Mistral model
2023-12-18 16:58:13 INFO semantic_router.utils.logger AI message:  How can I help you today?


' How can I help you today?'